In [1]:
import pandas as pd
import numpy as np
import jieba
from jieba import analyse
from gensim.models import Word2Vec, KeyedVectors

In [2]:
k_word1  = np.loadtxt( 'keyword1.txt', dtype=str, delimiter=',')
k_word2 = np.loadtxt( 'keyword2.txt', dtype=str, delimiter=',')

In [3]:
token1 = np.loadtxt( 'token1.txt', dtype=str, delimiter=',')
token2 = np.loadtxt( 'token2.txt', dtype=str, delimiter=',')

In [4]:
s0 = pd.read_csv( "sentence_token0.txt", header=None)
sentence_token0 = []
for string in s0[0]:
    string = string.split(' ')
    sentence_token0.append(string)
s1 = pd.read_csv( "sentence_token1.txt", header=None)
sentence_token1 = []
for string in s1[0]:
    string = string.split(' ')
    sentence_token1.append(string)
s2 = pd.read_csv( "sentence_token2.txt", header=None)
sentence_token2 = []
for string in s2[0]:
    string = string.split(' ')
    sentence_token2.append(string)

### Segmenting sentences(lists of token) input Word2Vec
#### 刪去stopword(1)的Segmenting sentences
1.建立詞彙庫，訓練Word2Vec模型

In [9]:
model1 = Word2Vec(window=5, min_count=3, sg=0)
model1.build_vocab(sentence_token1)
model1.train(sentence_token1, total_examples=model1.corpus_count, epochs=model1.epochs)

(1504, 5875)

2.觀察相似詞、相似度

In [38]:
for string in k_word1:
    print(string, end='\t')
    print(model1.wv.most_similar(string, topn=4))

野餐墊	[('地板', 0.18979725241661072), ('地方', 0.18961328268051147), ('100', 0.18448631465435028), ('報名', 0.18376493453979492)]
比較	[('搞錯', 0.3092630207538605), ('扭蛋', 0.2829122543334961), ('專業', 0.23818573355674744), ('西瓜汁', 0.20210972428321838)]
攤位	[('二手', 0.3528869152069092), ('925', 0.3047192692756653), ('吊衣桿', 0.25215768814086914), ('搞錯', 0.22324945032596588)]
衣架	[('扭蛋', 0.31511783599853516), ('報名', 0.3138881325721741), ('西瓜汁', 0.18614870309829712), ('兩', 0.1823175996541977)]
價錢	[('載', 0.2694505453109741), ('同一', 0.23577740788459778), ('超', 0.20908087491989136), ('晚上', 0.17758174240589142)]
標價	[('賣', 0.25317469239234924), ('地上', 0.22728179395198822), ('搞錯', 0.20731547474861145), ('那麼', 0.17904187738895416)]
衣服	[('右邊', 0.29106640815734863), ('要賣', 0.23079833388328552), ('沒', 0.21970200538635254), ('家', 0.21809600293636322)]
桌子	[('可愛', 0.31918826699256897), ('地上', 0.23941203951835632), ('錢', 0.21490682661533356), ('二手', 0.2047150582075119)]
過去	[('行李箱', 0.2237451672554016), ('錢', 0.18042570

3.期望能重組k_word1的詞，刪除不在k_word1的相似詞

In [11]:
s_word1 = {}
for string in k_word1:
    a = model1.wv.most_similar(string, topn=4)
    l = []
    for i in range(len(a)):
        if a[i][0] in k_word1:
            l.append(a[i][0])
    if len(l)>0:
        s_word1.update({string: l})
s_word1

{'野餐墊': ['地板', '地方'],
 '攤位': ['二手', '925', '吊衣桿'],
 '價錢': ['同一'],
 '衣服': ['右邊'],
 '桌子': ['可愛', '二手'],
 '過去': ['行李箱'],
 '吊衣桿': ['攤位', '二手', '桌布'],
 '和室桌': ['市集'],
 '可愛': ['桌子', '925'],
 '924': ['市集'],
 '925': ['攤位', '衣服'],
 '二手': ['攤位'],
 '市集': ['924', '和室桌', '討論'],
 '行李箱': ['地方', '啊啊', '過去'],
 '地板': ['野餐墊'],
 '一起': ['弟弟'],
 '公館': ['地方'],
 '時候': ['紙袋'],
 '啊啊': ['行李箱'],
 '右邊': ['衣服'],
 '101': ['吊衣桿'],
 '紙袋': ['時候'],
 '拿出來': ['知道'],
 '多少': ['地方'],
 '地方': ['多少', '行李箱', '公館'],
 '同一': ['價錢', '桌布'],
 '意思': ['200'],
 '200': ['右邊', '924'],
 '知道': ['拿出來'],
 '目前': ['924']}

4.一個分詞的相似字，帶入Word2Vec計算出相似字，三個詞在詞向量的距離都很近，可以連接組成句子。  
．s_word1字典中，keys為尋找相似字的分詞，values為尋找到的相似字。連接k、values可以組成句子。  
．下面兩個function找出同時為keys、values的分詞：  
findwordkey一橫排為單位將相似字對比keys，搜尋keys中的相同分詞。找到key連結的values，合併到該相似字list中。  
findwordvalue一橫排為單位將相似字對比其他keys的相似字，搜尋values中的相同分詞。找到values連結的key一起合併為一個list。  
．若找到的分詞為重複，則不加入list，保留前面與keys相似度較高的list位置。

In [12]:
def findwordkey(key, word, sdict):
    if word in sdict.keys() and len(sdict[word])==1 and sdict[word][0] in [key]:
        return False
    elif word in sdict.keys():
        l=[]
        for s in sdict[word]:
            if s not in [key]:
                l.append(s)
        return l
    else:
        return False
def findwordvalue(key, word, sdict):
    l=[]
    for k, v in sdict.items():
        if word in v and k not in [key]:
            l.append(k)
    if len(l)==0:
        return False
    else:
        return l

In [13]:
before1 = {}
for k, v in s_word1.items():
    l=[]
    for s in v:
        if s not in l:
            l.append(s)
        if findwordkey(k, s, s_word1) != False :
            l0=findwordkey(k, s, s_word1)
            for s0 in l0:
                if s0 not in l:
                    l.append(s0)
        if findwordvalue(k, s, s_word1) != False :
            l0=findwordvalue(k, s, s_word1)
            for s0 in l0:
                if s0 not in l:
                    l.append(s0)
    if len(l)>0:
        before1.update({k: l})

In [14]:
before1

{'野餐墊': ['地板', '地方', '多少', '行李箱', '公館'],
 '攤位': ['二手', '桌子', '吊衣桿', '925', '衣服', '可愛', '桌布', '101'],
 '價錢': ['同一', '桌布'],
 '衣服': ['右邊', '200'],
 '桌子': ['可愛', '925', '二手', '攤位', '吊衣桿'],
 '過去': ['行李箱', '地方', '啊啊'],
 '吊衣桿': ['攤位', '二手', '925', '桌子', '桌布', '同一'],
 '和室桌': ['市集', '924', '討論'],
 '可愛': ['桌子', '二手', '925', '攤位', '衣服'],
 '924': ['市集', '和室桌', '討論'],
 '925': ['攤位', '二手', '吊衣桿', '衣服', '右邊'],
 '二手': ['攤位', '925', '吊衣桿'],
 '市集': ['924', '200', '目前', '和室桌', '討論'],
 '行李箱': ['地方', '多少', '公館', '野餐墊', '啊啊', '過去'],
 '地板': ['野餐墊', '地方'],
 '一起': ['弟弟'],
 '公館': ['地方', '多少', '行李箱', '野餐墊'],
 '時候': ['紙袋'],
 '啊啊': ['行李箱', '地方', '過去'],
 '右邊': ['衣服', '925'],
 '101': ['吊衣桿', '攤位', '二手', '桌布'],
 '紙袋': ['時候'],
 '拿出來': ['知道'],
 '多少': ['地方', '行李箱', '公館', '野餐墊'],
 '地方': ['多少', '行李箱', '啊啊', '過去', '公館'],
 '同一': ['價錢', '桌布', '吊衣桿'],
 '意思': ['200', '右邊', '924'],
 '200': ['右邊', '衣服', '924', '市集', '目前'],
 '知道': ['拿出來'],
 '目前': ['924', '市集', '200']}

5.找出最長的list，組成句子

In [18]:
calculatemax = []
for v in before1.values():
    calculatemax.append(len(v))
for i, key in enumerate(before1):
    if i==calculatemax.index(max(calculatemax)):
        sentence={key: before1[key]}
sentence

{'攤位': ['二手', '桌子', '吊衣桿', '925', '衣服', '可愛', '桌布', '101']}

．未來可以利用CBOW找出更多相似字，組合為更長的摘要

In [41]:
model1.predict_output_word(['攤位', '二手', '桌子', '吊衣桿', '桌布'], topn=10)

[('袋子', 0.007874426),
 ('買下來', 0.007874355),
 ('兩', 0.007874296),
 ('賣', 0.00787429),
 ('錢', 0.0078742625),
 ('市集', 0.007874249),
 ('讚', 0.007874235),
 ('辣', 0.007874232),
 ('手作', 0.007874212),
 ('拍', 0.00787417)]

In [46]:
model1.predict_output_word(['桌子', '吊衣桿', '衣服', '桌布'], topn=10)

[('袋子', 0.007874275),
 ('地上', 0.007874259),
 ('買下來', 0.0078742495),
 ('錢', 0.00787424),
 ('辣', 0.007874234),
 ('擠', 0.00787421),
 ('100', 0.0078742),
 ('924', 0.007874198),
 ('真假', 0.00787419),
 ('小燈', 0.007874187)]

#### 刪去stopword(2.3)的Segmenting sentences

In [20]:
model2 = Word2Vec(window=5, min_count=3, sg=1)
model2.build_vocab(sentence_token2)
model2.train(sentence_token2, total_examples=model2.corpus_count, epochs=model2.epochs)

(1699, 6230)

In [21]:
for string in k_word2:
    print(string, end='')
    print(model2.wv.most_similar(string, topn=4))

野餐墊[('討論', 0.19075395166873932), ('姚', 0.19064442813396454), ('搶', 0.18721728026866913), ('行李箱', 0.18543832004070282)]
衣架[('六', 0.31160566210746765), ('衣桿', 0.28117233514785767), ('市集', 0.23571784794330597), ('方向', 0.2052355855703354)]
攤位[('可愛', 0.3173445165157318), ('小', 0.24104639887809753), ('那麼', 0.2189498394727707), ('好笑', 0.209580659866333)]
價錢[('好笑', 0.3500855565071106), ('這麼', 0.3033530116081238), ('載', 0.25184816122055054), ('六', 0.22767388820648193)]
標價[('怎樣', 0.26384198665618896), ('大家', 0.24807527661323547), ('袋子', 0.23632220923900604), ('小燈', 0.1965366154909134)]
這麼[('價錢', 0.30335304141044617), ('西瓜汁', 0.22224554419517517), ('衣服', 0.19832643866539001), ('確定', 0.19474099576473236)]
衣服[('六晚', 0.2910084128379822), ('西瓜汁', 0.23170271515846252), ('過去', 0.2293713092803955), ('懂', 0.2139437347650528)]
過去[('最近', 0.23039737343788147), ('衣服', 0.22937126457691193), ('長', 0.22646880149841309), ('市集', 0.20562860369682312)]
桌子[('便宜', 0.2685784697532654), ('拍', 0.23640720546245575), ('以為

In [22]:
s_word2 = {}
for string in k_word2:
    a = model2.wv.most_similar(string, topn=4)
    l = []
    for i in range(len(a)):
        if a[i][0] in k_word2:
            l.append(a[i][0])
    if len(l)>0:
        s_word2.update({string: l})
s_word2

{'野餐墊': ['討論', '行李箱'],
 '衣架': ['市集'],
 '攤位': ['可愛', '好笑'],
 '價錢': ['好笑', '這麼'],
 '標價': ['大家'],
 '這麼': ['價錢', '衣服', '確定'],
 '衣服': ['六晚', '過去'],
 '過去': ['衣服', '市集'],
 '桌子': ['便宜'],
 '吊衣桿': ['弟弟', '這麼'],
 '和室桌': ['六晚'],
 '可愛': ['攤位', '這麼', '標價'],
 '討論': ['如果', '不行', '有沒有'],
 '狀況': ['右邊', '標價'],
 '六晚': ['衣服', '右邊', '拿出來'],
 '925': ['多少', '行李箱', '雖然'],
 '二手': ['不行'],
 '市集': ['雖然', '衣架', '好笑'],
 '行李箱': ['925'],
 '地板': ['有沒有', '標價'],
 '如果': ['討論', '確定'],
 '大家': ['標價', '紙袋'],
 '那邊': ['不行', '右邊', '六晚'],
 '沒錯': ['有沒有', '他們'],
 '好了': ['和室桌', '六晚'],
 '公館': ['和室桌', '大概', '好了'],
 '啊啊': ['有沒有'],
 '雖然': ['市集', '925'],
 '右邊': ['六晚', '那邊'],
 '101': ['和室桌', '那邊'],
 '確定': ['如果', '這麼'],
 '紙袋': ['大家'],
 '拿出來': ['六晚'],
 '200': ['紙袋', '不用'],
 '有沒有': ['沒錯', '討論', '地板'],
 '他們': ['沒錯', '便宜'],
 '好笑': ['價錢', '多少'],
 '多少': ['好笑', '925'],
 '方便': ['地方'],
 '地方': ['方便'],
 '弟弟': ['吊衣桿', '衣架'],
 '便宜': ['桌子', '他們'],
 '不行': ['討論', '那邊'],
 '大概': ['好笑']}

In [23]:
before2 = {}
for k, v in s_word2.items():
    l=[]
    for s in v:
        if s not in l:
            l.append(s)
        if findwordkey(k, s, s_word2) != False :
            l0=findwordkey(k, s, s_word2)
            for s0 in l0:
                if s0 not in l:
                    l.append(s0)
        if findwordvalue(k, s, s_word2) != False :
            l0=findwordvalue(k, s, s_word2)
            for s0 in l0:
                if s0 not in l:
                    l.append(s0)
    if len(l)>0:
        before2.update({k: l})

In [24]:
before2

{'野餐墊': ['討論', '如果', '不行', '有沒有', '行李箱', '925'],
 '衣架': ['市集', '雖然', '好笑', '過去'],
 '攤位': ['可愛', '這麼', '標價', '好笑', '價錢', '多少', '市集', '大概'],
 '價錢': ['好笑', '多少', '攤位', '市集', '大概', '這麼', '衣服', '確定', '吊衣桿', '可愛'],
 '標價': ['大家', '紙袋'],
 '這麼': ['價錢', '好笑', '衣服', '六晚', '過去', '確定', '如果'],
 '衣服': ['六晚', '右邊', '拿出來', '和室桌', '那邊', '好了', '過去', '市集'],
 '過去': ['衣服', '六晚', '這麼', '市集', '雖然', '衣架', '好笑'],
 '桌子': ['便宜', '他們'],
 '吊衣桿': ['弟弟', '衣架', '這麼', '價錢', '衣服', '確定', '可愛'],
 '和室桌': ['六晚', '衣服', '右邊', '拿出來', '那邊', '好了'],
 '可愛': ['攤位', '好笑', '這麼', '價錢', '衣服', '確定', '吊衣桿', '標價', '大家', '狀況', '地板'],
 '討論': ['如果', '確定', '不行', '那邊', '二手', '有沒有', '沒錯', '地板', '啊啊'],
 '狀況': ['右邊', '六晚', '那邊', '標價', '大家', '可愛', '地板'],
 '六晚': ['衣服', '過去', '這麼', '右邊', '那邊', '狀況', '拿出來'],
 '925': ['多少', '好笑', '行李箱', '野餐墊', '雖然', '市集'],
 '二手': ['不行', '討論', '那邊'],
 '市集': ['雖然', '925', '衣架', '弟弟', '好笑', '價錢', '多少', '攤位', '大概'],
 '行李箱': ['925', '多少', '雖然'],
 '地板': ['有沒有', '沒錯', '討論', '啊啊', '標價', '大家', '可愛', '狀況'],
 '如果': ['討論', '不行', 

In [25]:
calculatemax = []
for v in before2.values():
    calculatemax.append(len(v))
for i, key in enumerate(before2):
    if i==calculatemax.index(max(calculatemax)):
        sentence={key: before2[key]}
sentence

{'可愛': ['攤位', '好笑', '這麼', '價錢', '衣服', '確定', '吊衣桿', '標價', '大家', '狀況', '地板']}

#### 沒設置停用詞(原始訊息)的Segmenting sentences  
．找出的相似詞通常為停用詞

In [27]:
model0 = Word2Vec(window=5, min_count=2, workers=4, sg=1)
model0.build_vocab(sentence_token0, progress_per=1000)
model0.train(sentence_token0, total_examples=model0.corpus_count, epochs=model0.epochs)

(7890, 14575)

In [28]:
for string in k_word1:
    print(string, end='')
    print(model0.wv.most_similar(string, topn=1))

野餐墊[('在', 0.9584750533103943)]
比較[('可以', 0.9661309719085693)]
攤位[('的', 0.9813362956047058)]
衣架[('', 0.9713442325592041)]
價錢[('', 0.9683171510696411)]
標價[('的', 0.9760774374008179)]
衣服[('', 0.971947193145752)]
桌子[('我', 0.9734164476394653)]
過去[('我', 0.9543203711509705)]
桌布[('', 0.9657148122787476)]
吊衣桿[('', 0.9066082239151001)]
和室桌[('我', 0.9234816431999207)]
可愛[('我', 0.9352328777313232)]
時間[('', 0.9528123140335083)]
討論[('一起', 0.8934841752052307)]
擺攤[('', 0.919335663318634)]
924[('的', 0.9292513728141785)]
925[('那', 0.919966459274292)]
二手[('', 0.9728482961654663)]
市集[('', 0.9559139609336853)]
行李箱[('', 0.9273357391357422)]
地板[('', 0.9508557319641113)]
一起[('', 0.9831023216247559)]
公館[('對', 0.8538476824760437)]
時候[('哈哈', 0.8474107980728149)]
啊啊[('桌布', 0.5578004717826843)]
右邊[('的', 0.9033402800559998)]
101[('ㄚ', 0.7159742116928101)]
紙袋[('大家', 0.9254161715507507)]
拿出來[('有', 0.7968002557754517)]
多少[('的', 0.9481890201568604)]
方便[('大家', 0.9207934141159058)]
地方[('', 0.9618049263954163)]
弟弟[('我', 0.9

In [30]:
for string in k_word2:
    print(string, end='')
    print(model0.wv.most_similar(string, topn=1))

野餐墊[('在', 0.9584750533103943)]
衣架[('', 0.9713442325592041)]
攤位[('的', 0.9813362956047058)]
價錢[('', 0.9683171510696411)]
標價[('的', 0.9760774374008179)]
這麼[('', 0.9599199295043945)]
衣服[('', 0.971947193145752)]
過去[('我', 0.9543203711509705)]
桌子[('我', 0.9734164476394653)]
桌布[('', 0.9657148122787476)]
吊衣桿[('', 0.9066082239151001)]
和室桌[('我', 0.9234816431999207)]
可愛[('我', 0.9352328777313232)]
討論[('一起', 0.8934841752052307)]
擺攤[('', 0.919335663318634)]
狀況[('', 0.9372735023498535)]
六晚[('桌布', 0.8138114809989929)]
924[('的', 0.9292513728141785)]
925[('那', 0.919966459274292)]
二手[('', 0.9728482961654663)]
市集[('', 0.9559139609336853)]
行李箱[('', 0.9273357391357422)]
不用[('', 0.9539898037910461)]
地板[('', 0.9508557319641113)]
如果[('', 0.9787666201591492)]
大家[('', 0.9691208600997925)]
那邊[('衣架', 0.8621832728385925)]
沒錯[('好像', 0.7693094611167908)]
好了[('的', 0.93317711353302)]
公館[('對', 0.8538476824760437)]
啊啊[('桌布', 0.5578004717826843)]
雖然[('家', 0.8872878551483154)]
右邊[('的', 0.9033402800559998)]
101[('ㄚ', 0.7159742

In [44]:
model0.predict_output_word(['桌子', '吊衣桿', '衣服', '桌布', '101'], topn=10)

[('下面', 0.0028230585),
 ('進去', 0.0028212846),
 ('椅子', 0.00281933),
 ('困住', 0.002816321),
 ('家放', 0.002815118),
 ('著', 0.0028141255),
 ('飾品', 0.0028139912),
 ('沒關係', 0.002813795),
 ('直接', 0.0028137772),
 ('想要', 0.0028137474)]